In [27]:
import random
import easygui as g


In [28]:
wuerfel = []
wuerfel.append([random.randint(1, 6) for i in range(0, 5)])
wuerfel = wuerfel[0]
print(sorted(wuerfel))


[1, 3, 5, 5, 5]


In [29]:
#wuerfel=[1,2,3,4,5]

In [30]:
classCheck = [0, 0, 0, 0, 0, 0, 0, 0]
classes = (
    "P",  # Augenzahl (hier Paare um aus 3er Pasch und Paar ein Full House zu finden)
    "3P",  # 3er Pasch
    "4P",  # 4er Pasch
    "F",  # Full House
    "KS",  # Kleine Straße
    "GS",  # Große Straße
    "K",  # Kniffel
    "C",  # Chance
    "ST" # Streichen
)
classDict = dict(zip(classes, classCheck))
print(sorted(wuerfel))


[1, 3, 5, 5, 5]


In [31]:
wuerfel.sort()
paare = []
playerClassesUsed = [[[0], [0], [0], [0], [0], [0]], [[0], [0], [0], [0], [0], [0], [0]]]
for l in range(1, 7):

    for i in range(0, 5):  # 1. Stelle i

        for j in range(0, 5):  # 2. Stelle j
            if j != i:

                for k in range(0, 5):  # 3. Stelle k
                    if k != j and k != i:

                        # 3er Pasch ----------------------------------
                        if (
                            wuerfel[i] == wuerfel[j] == wuerfel[k]
                            and classDict["3P"] == 0
                        ):
                            # Speichert die 3er Paschzahl in classCheck ab
                            classCheck[1] = wuerfel[i]
                            # Setzt 3er Pasch von 0 auf 1 (False -> True)
                            classDict["3P"] = 1
                            playerClassesUsed[1][0][0]=1

                        # Paare für Fullhouse schauen ----------------------
                        if (
                            wuerfel[i] == wuerfel[j]  # Beide Würfel vergleichen
                            # Schauen ob bereits ein Eintrag in classDict gemacht wurde <2 weil zwei Paare möglich sind
                            and classDict["P"] < 2
                            # Schaut ob erstes gefundenes Paar ungleich dem Würfel ist
                            and classCheck[0] != wuerfel[i]
                            # Schaut ob aktueller Würfel nicht die Zahl des Pasch ist
                            and classCheck[1] != wuerfel[i]
                            # Erst ab l>2 anfangen nach Paaren zu suchen, weil sonst Paare vor Pasche gefunden werden und doppelt vorkommen
                            and l > 1
                        ):
                            # Speichert erstes paar in classCheck ab
                            classCheck[0] = wuerfel[i]
                            # Speichert Paare in Liste, damit am Ende alle Paare in classCheck eingefügt werden können
                            paare.append(wuerfel[i])
                            # classDict += weil es hierbei zwei mögliche Paare gibt und nicht nur eins wie bei 3er oder 4er Pasch
                            classDict["P"] += 1
                            

                        for m in range(0, 5):  # 4. Stelle m

                            if m != j and m != i and m != k:
                                # 4er Pasch ----------------------------------
                                if (
                                    wuerfel[i] == wuerfel[j] == wuerfel[k] == wuerfel[m]
                                    and classDict["4P"] == 0
                                ):
                                    # Speichert die 4er Paschzahl in classCheck ab
                                    classCheck[2] = wuerfel[i]
                                    # Setzt 4er Pasch von 0 auf 1 (False -> True)
                                    classDict["4P"] = 1
                                    playerClassesUsed[1][1][0]=1
                                # Kleine Straße ------------------------------
                                if (
                                    [wuerfel[i], wuerfel[j], wuerfel[k], wuerfel[m]]
                                    == [1, 2, 3, 4]
                                    or [wuerfel[i], wuerfel[j], wuerfel[k], wuerfel[m]]
                                    == [2, 3, 4, 5]
                                    or [wuerfel[i], wuerfel[j], wuerfel[k], wuerfel[m]]
                                    == [3, 4, 5, 6]
                                    and classDict["KS"] != 1
                                ):
                                    classDict["KS"] = 1
                                    playerClassesUsed[1][3][0]=1

                                for n in range(0, 5):  # 5. Stelle n

                                    if n != j and n != i and n != k and n != m:
                                        # Kniffel ----------------------------------
                                        if (
                                            wuerfel[i]
                                            == wuerfel[j]
                                            == wuerfel[k]
                                            == wuerfel[m]
                                            == wuerfel[n]
                                            and classDict["K"] == 0
                                        ):
                                            # Setzt Kniffel von 0 auf 1 (False -> True)
                                            classDict["K"] == 1
                                            playerClassesUsed[1][5][0]+=1

                                    # Große Straße ------------------------------
                                    if (
                                        [
                                            wuerfel[i],
                                            wuerfel[j],
                                            wuerfel[k],
                                            wuerfel[m],
                                            wuerfel[n],
                                        ]
                                        == [1, 2, 3, 4, 5]
                                        or [
                                            wuerfel[i],
                                            wuerfel[j],
                                            wuerfel[k],
                                            wuerfel[m],
                                            wuerfel[n],
                                        ]
                                        == [2, 3, 4, 5, 6]
                                        and classDict["GS"] != 1
                                    ):
                                        classDict["GS"] = 1
                                        playerClassesUsed[1][4][0]=1

# Full House ----------------------------------
if classDict["3P"] == 1 and classDict["P"] >= 1:
    classDict["F"] = 1
    playerClassesUsed[1][2][0]=1

# Chance ----------------------------------
if (
    classDict["3P"] == 0
    and classDict["4P"] == 0
    and classDict["F"] == 0
    and classDict["KS"] == 0
    and classDict["GS"] == 0
    and classDict["K"] == 0
):
    classDict["C"] = 1
    playerClassesUsed[1][6][0]=1

classCheck[0] = paare
print(
    f"3er Pasch mit Zahl: {classCheck[1]}\n" if classDict["3P"] == 1 else "\n",
    f"4er Pasch mit Zahl: {classCheck[2]}\n" if classDict["4P"] == 1 else "\n",
    f"Full House\n" if classDict["F"] == 1 else "\n",
    f"Kleine Straße\n" if classDict["KS"] == 1 else "\n",
    f"Große Straße\n" if classDict["GS"] == 1 else "\n",
    f"KNIFFEL!!!" if classDict["K"] == 1 else "",
    f"Chance" if classDict["C"] == 1 else "",
)



3er Pasch mit Zahl: 5
 
 
 
 
  


In [36]:
# Auswahl
classes = (
    "Augenzahl",
    "3er Pasch",
    "4er Pasch",
    "Full House",
    "Kleine Straße",
    "Große Straße",
    "Kniffel",
    "Chance",
    "Streichen",
)

# ---------- Variablen für Auswahl ----------
# Classes Used aus Player Klasse
playerClassesUsed = [
    [[0], [0], [1], [0], [1], [0]],
    [[0], [0], [0], [0], [0], [2], [0]],
]
doppelkniffel = [[[0], [0], [0], [0], [0], [0]], [[0], [0], [0], [0], [0], [0], [0]]]
augenzahlen = [1, 2, 3, 4, 5, 6]  # Neu
auswahlAugen = []  # Neu
auswahlSpecials = []  # Neu
auswahlDoppelKniffel = []
classDictKeys = list(classDict)  # Keys von classDict als Liste
istDoppelkniffel = False
alleAugenDK = []
alleSpecialDK = []
alleDoppelkniffel = []

if playerClassesUsed[1][5][0] > 1:
    istDoppelkniffel = True

# ---------- Auswahl vorbereiten ----------
# Augenzahlen in Auswahl stecken
for i in range(0, 6):
    alleAugenDK.append([f"Doppelkniffel für {i+1}",i+1])

    if playerClassesUsed[0][i] == [0]:
        auswahlAugen.append(i + 1)

    # Doppelkniffel
    if playerClassesUsed[0][i] == [0] and istDoppelkniffel:
        auswahlDoppelKniffel.append(f"Doppelkniffel für Augenzahl: {i+1}")



# Specials in Auswahl stecken
for i in range(0, len(classDict) - 1):
    # Specials in alle möglichen Doppelkniffel hinzufügen
    alleSpecialDK.append([f"Doppelkniffel für {classes[i + 1]}",i+7])

    if classDict[classDictKeys[i + 1]] == 1 and playerClassesUsed[1][i][0] == 0:
        auswahlSpecials.append(classes[i + 1])
    # Doppelkniffel in Auswahl
    if (
        classDict[classDictKeys[i + 1]] == 1
        and playerClassesUsed[1][i][0] == 0
        and istDoppelkniffel
        and i != 5
    ):
        auswahlDoppelKniffel.append(f"Doppelkniffel für {classes[i+1]}")

# Doppelkniffel zurücksetzen
if len(auswahlDoppelKniffel) > 0:
    playerClassesUsed[1][5][0] = 1

# ---------- Auswahl ----------
# Gesamtauswahl
gesamtauswahl = auswahlAugen + auswahlSpecials + auswahlDoppelKniffel

while True:
    antwort = g.choicebox(
        msg="Treffe deine Entscheidung für diesen Spielzug",
        choices=gesamtauswahl,
        title="Auswahl",
        preselect=0,
    )
    if antwort == None:
        print("Keine gültige Eingabe")
        g.msgbox(msg="Keine gültige Eingabe", title="Fehler", ok_button="Nochmal")
    else:
        break

# ---------- Auswahl verarbeiten ----------
# Entweder Index des Spezialwurf nehmen oder Antwort ist Augenzahl oder Antwort ist Doppelkniffel
antwortIndexSpecials = None
antwortIndexAugen = None
antwortIndexKniffel = None
# Indexsuche von Spezialwürfen als Antwort
for i in range(0, len(classes)):

    if classes[i] == antwort:
        antwortIndexSpecials = i

# Indexsuche von Augenzahl als Antwort
if isinstance(antwort,int):
    antwortIndexAugen = antwort

# Indexsuche wenn Antwort ein Doppelkniffel ist
alleDoppelkniffel=alleAugenDK+alleSpecialDK
if antwortIndexAugen == None:
    print(alleDoppelkniffel)
    for i in range(0, len(alleDoppelkniffel)):
        print(i)

print(f"Antwort: {antwort}, Type:{type(antwort)}")


[['Doppelkniffel für 1', 1], ['Doppelkniffel für 2', 2], ['Doppelkniffel für 3', 3], ['Doppelkniffel für 4', 4], ['Doppelkniffel für 5', 5], ['Doppelkniffel für 6', 6], ['Doppelkniffel für 3er Pasch', 7], ['Doppelkniffel für 4er Pasch', 8], ['Doppelkniffel für Full House', 9], ['Doppelkniffel für Kleine Straße', 10], ['Doppelkniffel für Große Straße', 11], ['Doppelkniffel für Kniffel', 12], ['Doppelkniffel für Chance', 13]]
0
1
2
3
4
5
6
7
8
9
10
11
12
Antwort: Doppelkniffel für Augenzahl: 6, Type:<class 'str'>
